In [1]:
import pandas as pd

In [2]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [3]:
PATH = 'database/data.feather'

In [4]:
df = pd.read_feather(PATH)
print(df.head())

   age  air_yards  ast  away  aypt        college  day  experience  ffum  \
0   32        0.0    0     0   0.0    Mississippi    5          10     0   
1   25       39.0    0     0   6.5      Texas A&M    5           6     0   
2   26       88.0    0     0   8.0  Massachusetts    5           3     0   
3   24        0.0    1     0   0.0  South Florida    5           4     0   
4   24        0.0    0     0   0.0  East Carolina    5           4     0   

   firstname   ...    tot tot_score  trcv  weekday  weight  wopr   yac  yds  \
0        Eli   ...    0.0        17   0.0        2      99  0.00   0.0  0.0   
1  Martellus   ...    0.0        17   0.0        2     124  0.38  24.0  0.0   
2     Victor   ...    0.0        17   0.0        2      91  0.73  16.0  0.0   
3      Jason   ...    0.0        17   0.0        2     124  0.00   0.0  0.0   
4     Linval   ...    0.0        17   0.0        2     149  0.00   0.0  0.0   

   year  season  
0  2012    2012  
1  2012    2012  
2  2012    201

In [5]:
def split_frame(df):
    '''splits a dataframe in continous and categorial part'''
    
    cont_vars = ['air_yards', 'ast', 'aypt', 'ffum', 'k_xpb', 'k_xpmade',
            'k_xpmissed', 'k_xptot', 'int', 'k_fga', 'k_fgm', 'k_fgyds', 
            'k_xpa', 'k_totpfg', 'kret_avg', 'kret_lng', 'kret_lngtd', 'kret_ret', 'kret_tds', 'lost', 'ms_air_yards', 'p_avg', 'p_i20',
            'p_lng', 'pass_twopta', 'pass_twoptm', 'pass_cmp', 'pass_ints', 'pass_tds',
            'pass_att', 'p_yds', 'p_pts', 'pass_yds', 'pret_avg', 'pret_lng', 'pret_lngtd',
            'pret_ret', 'pret_tds', 'pts_allwd', 'racr', 'rcv', 'rcv_lng', 'rcv_lngtd',
            'rcv_rec', 'rcv_tds', 'rcv_twopta', 'rcv_twoptm', 'rcv_yds', 'rush_att',
            'rush_lng', 'rush_lngtd', 'rush_tds', 'rush_twopta', 'rush_twoptm',
            'rush_yds', 'sk', 'tar', 'target_share', 'tkl', 'yac', 'tm_airyards', 
            'tot', 'tot_score', 'trcv', 'wopr', 'yac', 'yds']
    cat_vars = set(df.columns) - set(cont_vars)
    ContFrame = pd.DataFrame()
    CatFrame = pd.DataFrame()
    for item in cont_vars:
        ContFrame[item] = df[item]
    for item in cat_vars:
        CatFrame[item] = df[item]
    return ContFrame, CatFrame

In [6]:
def calculate_ewm(df, alph=0.4):
    df = df.fillna(0)
    m = df.ewm(alpha=alph).mean()
    m = m.shift(1, axis=0)
    m = m.fillna(0)
    return m

In [7]:
seasons = set(df.season)
players = set(df.full_name)
frames = []
for s in seasons:
    df2 = df[df.season == s].copy()
    for player in players:
        df3 = df2[df2.full_name == player]
        df3 = df3.sort_values(by='seasonweek')
        cont, cat = split_frame(df3)
        cont = calculate_ewm(cont)
        df3 = pd.concat([cat, cont], axis=1)
        frames.append(df3)
result = pd.concat(frames)
#result.to_feather('database/processed_data.feather')
result.to_csv('database/processed_data.csv')
result.head()

,position,season,fpts,firstname,college,seasonweek,playerID,height,experience,weight,...,tar,target_share,tkl,yac,tm_airyards,tot,tot_score,trcv,wopr,yds
33072,LB,2016,2.076923,James,Kent State,1,00-0020712,182,15,109,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
33489,LB,2016,0.062500,James,Kent State,2,00-0020712,182,15,109,...,0.0,0.0,2.000000,0.0,0.0,0.0,38.000000,0.0,0.0,0.0
34510,LB,2016,-0.062500,James,Kent State,3,00-0020712,182,15,109,...,0.0,0.0,1.375000,0.0,0.0,0.0,29.250000,0.0,0.0,0.0
35370,LB,2016,0.076923,James,Kent State,4,00-0020712,182,15,109,...,0.0,0.0,2.204082,0.0,0.0,0.0,15.857143,0.0,0.0,0.0
35801,LB,2016,0.266667,James,Kent State,5,00-0020712,182,15,109,...,0.0,0.0,2.110294,0.0,0.0,0.0,28.330882,0.0,0.0,0.0


In [8]:
result = pd.concat(frames)
result = result.sort_index()
result.to_feather('database/processed_data.feather')
result.to_csv('database/processed_data.csv')
result.head()

,position,season,fpts,firstname,college,seasonweek,playerID,height,experience,weight,...,tar,target_share,tkl,yac,tm_airyards,tot,tot_score,trcv,wopr,yds
0,QB,2012,12.52,Eli,Mississippi,1,00-0022803,195,10,99,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,TE,2012,10.00,Martellus,Texas A&M,1,00-0026201,198,6,124,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,WR,2012,5.80,Victor,Massachusetts,1,00-0027265,182,3,91,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,DE,2012,0.00,Jason,South Florida,1,00-0027867,195,4,124,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,DT,2012,1.00,Linval,East Carolina,1,00-0027885,193,4,149,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [37]:
len(result) == len(df)

True

In [38]:
len(result.columns) == len(df.columns)

True

False